# Introduction

In this competition, you are required to recognize landmark present in test images. The landmark recognition challenge contains more than 81K classes, available across train folders with their labels provided in a csv file. Each image has a unique image id.

This notebook contains EDA to explore the folder structure, build visuals of the image distributions and understand the classes and labels.

In [ ]:
# Import libraries 
import pandas as pd
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import PIL
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

init_notebook_mode()

## Table of contents

1. [Exploring the folder structures](#folder)
2. [Image data consolidation](#labelsconsolidate)
3. [Looking at the images](#look)
4. [Plots of label frequencies](#frequencies)
5. [Plots of label distribution](#distributions)
6. [Image dimensions](#dimensions)

## Folder structure <a class="anchor" id="folders"> </a>

As a first step to understanding the data structures, folders and filepaths for this competition, lets look at the folder structure for the train and test data images.We'll build directories for each relevant folder path.

The [train] folder containing the images is stored across 16 folders, with names 0-9, a-f.
Each of these folders is further a series of nested folders, as below: 
<code>

|---0
    |---0
        |---0
            |---0000059611c7d079.jpg
            |---00...............jpg
        |---1
    |---1
    |---2
|---1
</code>


Let's assign the folder paths to variables.

In [ ]:
print(f"{'--'*20} \n LIST OF FOLDERS: \n {os.listdir(os.path.join('../'))} \n {'--'*20} \n")
print(f"LIST OF FOLDERS INSIDE [input] FOLDER: \n {os.listdir(os.path.join('../input'))} \n {'--'*20} \n")
print(f"LIST OF FOLDERS INSIDE [input/landmark-recognition-2020] FOLDER: \n {os.listdir(os.path.join('../input/landmark-recognition-2020'))} \n {'--'*20} \n")

# Assigning paths to variables
INPUT_PATH = os.path.join('..', 'input')
DATASET_PATH = os.path.join(INPUT_PATH, 'landmark-recognition-2020')
TRAIN_IMAGE_PATH = os.path.join(DATASET_PATH, 'train')
TEST_IMAGE_PATH = os.path.join(DATASET_PATH, 'test')
TRAIN_CSV_PATH = os.path.join(DATASET_PATH, 'train.csv')
SUBMISSION_CSV_PATH = os.path.join(DATASET_PATH, 'sample_submission.csv')

train_df = pd.read_csv(TRAIN_CSV_PATH)
print(f"{'--'*20} \n SNIPPET OF TRAINING DATA: \n {train_df.head()} \n {'--'*20} \n Number of rows in train data: {train_df.shape[0]} \n {'--'*20}")

submission_df = pd.read_csv(SUBMISSION_CSV_PATH)
print(f"{'--'*20} \n SNIPPET OF TEST DATA: \n {submission_df.head()} \n {'--'*20} \n Number of rows in test data: {submission_df.shape[0]} \n {'--'*20}")

print(f"EXAMPLE FOR LANDMARK-LABEL MAPPING FOR [17660ef415d37059.jpg] \n FOLDER STRUCTURE: \n |---1 \n \t |---7 \n \t \t |---6 \n \t \t \t|---<17660ef415d37059.jpg>")
i=0
print(f"Image name: {train_df['id'].iloc[i]}")
print(f"First folder to look inside: {train_df['id'][i][0]}")
print(f"Second folder to look inside: {train_df['id'][i][1]}")
print(f"Second folder to look inside: {train_df['id'][i][2]}")

## Consolidate label names <a class="anchor" id="labelsconsolidate"> </a>

Lets build a dataframe containing all filenames with their corresponding labels

In [ ]:
# Build dictionary to store image paths & labels
print(f"{'---'*20} \n Creating training data mapping \n {'---'*20}")
data_label_dict = {'image': [], 'target': []}
for i in tqdm(range(train_df.shape[0])):
    data_label_dict['image'].append(
        TRAIN_IMAGE_PATH + '/' +
        train_df['id'][i][0] + '/' + 
        train_df['id'][i][1]+ '/' +
        train_df['id'][i][2]+ '/' +
        train_df['id'][i] + ".jpg")
    data_label_dict['target'].append(
        train_df['landmark_id'][i])
#Convert to dataframe
train_pathlabel_df = pd.DataFrame(data_label_dict)
print(train_pathlabel_df.head())
    
print(f"{'---'*20} \n Creating test data mapping \n {'---'*20}")
data_label_dict = {'image': []}
for i in tqdm(range(submission_df.shape[0])):
    data_label_dict['image'].append(
        TEST_IMAGE_PATH + '/' +
        submission_df['id'][i][0] + '/' + 
        submission_df['id'][i][1]+ '/' +
        submission_df['id'][i][2]+ '/' +
        submission_df['id'][i] + ".jpg")

test_pathlabel_df = pd.DataFrame(data_label_dict)
print(test_pathlabel_df.head())

## Looking at the images <a class="anchor" id='look'> </a>

The training data has 81,313 unique classes of landmarks.

Lets take a look at the first 4 landmarks to understand how the images look, and get an idea of how many instances we have per image.

In [ ]:
print(f"The data has {train_pathlabel_df['target'].nunique()} unique classes")

for tar in train_pathlabel_df['target'].unique()[:4]: 
    #Subset to just that target 
    label_df = train_pathlabel_df[train_pathlabel_df['target']==tar].reset_index()
    cols = 2
    rows = 2
    fig = plt.figure(figsize = (4*cols - 1, 4.5*rows - 1))
    for c in range(cols):
        for r in range(rows):
            ax = fig.add_subplot(rows, cols, c*rows + r + 1)
            img = mpimg.imread(label_df['image'][c+r])
            ax.imshow(img)#label_df[][c+r])
    fig.suptitle(f"Images corresponding to label [{tar}] with a total of {label_df.shape[0]} images available")
    plt.show()
    plt.close()
    
    

## Plots of label frequencies <a class="anchor" id='plots'> </a> 

Lets look at how the count of images per landmark is distributed. First we'll build a dataframe of image counts per landmark, and then look at the frequency for the top 20 landmarks.

In [ ]:
counts_df = pd.DataFrame(train_df[['landmark_id']].value_counts().reset_index())
counts_df.columns = ['Landmark', 'Count']
counts_df.sort_values('Count', ascending=False, inplace=True)
print(counts_df.head())

fig = go.Figure(data = [go.Bar(x = counts_df[:20].index,
                              y=counts_df[:20]['Count'],
                              text=counts_df[:20]['Count'],
                              textposition = 'outside')])
fig.update_layout(title="Image counts across top 20 landmarks",
                 xaxis_title = "Landmark id",
                 yaxis_title = "Count of images")
fig.update_xaxes(ticktext= counts_df[:20]['Landmark'],
                tickvals = counts_df[:20].index)
fig.show()

It looks like the image classes are unbalanced, with 6272 images of `138982` category, followed by 2231 images of `126637` category. The smallest classes like `202176` and`183721` have as few as 2 images each.

Lets build some bins of the image counts to look at the number of images across different bins.

In [ ]:
counts_df['Bin'] = np.where((counts_df['Count']<=10), "(0, 10]", "Rest")
counts_df['Bin'] = np.where((counts_df['Count']>10) & (counts_df['Count']<=20), "(10, 20]", counts_df['Bin'])
counts_df['Bin'] = np.where((counts_df['Count']>20) & (counts_df['Count']<=30), "(20, 30]", counts_df['Bin'])
counts_df['Bin'] = np.where((counts_df['Count']>30) & (counts_df['Count']<=50), "(30, 50]", counts_df['Bin'])
counts_df['Bin'] = np.where((counts_df['Count']>50) & (counts_df['Count']<=70), "(50, 70]", counts_df['Bin'])
counts_df['Bin'] = np.where((counts_df['Count']>70) & (counts_df['Count']<=100), "(70, 100]", counts_df['Bin'])
counts_df['Bin'] = np.where((counts_df['Count']>100) & (counts_df['Count']<=150), "(100, 150]", counts_df['Bin'])
# counts_df['Bin'] = np.where((counts_df['Count']>=20) & (counts_df['Count']<30), "Bin 3: 20-30", counts_df['Bin'])
bin_df = counts_df.groupby('Bin')['Count'].count().reset_index()
bin_df['Bin'] = bin_df['Bin'].astype('str')
print(bin_df)

In [ ]:

fig = go.Figure(data = [go.Bar(x=bin_df['Bin'],
                               y=bin_df['Count'],
                               text = bin_df['Count'],
                              textposition = 'outside')])
fig.update_layout(title='Ímage counts across bins',
                  xaxis_title = "Bin/interval of image counts per landmark",
                 yaxis_title = "Count of images in bin")
# fig.update_xaxes('Bins/intervals of image counts')
fig.show()

We observe that:
* 44,646 landmarks have at least 10 images.
* Only 879 landmarks have more than 100 images

## Distribution of image labels <a class='anchor' id='distributions'> </a>

Let's look at how the target labels are distributed

In [ ]:
plt.figure(figsize = (15,10))
sns.distplot(train_pathlabel_df['target'], hist=True, bins=100)
plt.title('Distribution of image labels')

## Image dimensions <a class='anchor' id='dimensions'> </a>

For the first 50,000 images, lets look at a histogram of the heights & weights.

In [ ]:
dims_dict = {'image': [], 'width': [], 'height': [], 'channels': []}
for i in tqdm(range(50000)):#train_pathlabel_df['image'].unique())):
    dims = mpimg.imread(train_pathlabel_df['image'][i]).shape
    dims_dict['image'].append(train_pathlabel_df['image'][i])
    dims_dict['height'].append(dims[0])
    dims_dict['width'].append(dims[1])
    dims_dict['channels'].append(dims[2])

dims_df = pd.DataFrame(dims_dict).head()

sns.distplot(dims_df['height'])
plt.title('Distribution of image heights for first 50,000 images');

sns.distplot(dims_df['width'])
plt.title('Distribution of image widths for first 50,000 images');

That's all folks! Thanks for reading this far, this is my first Kaggle competition. In my next notebook I'll try some simple classifiers on the data.

Note: I am seeing my notebook displayed with some strange margins around markdown cells, the rendering doesn't look too pretty. I can't figure out why this is happening, would appreciate any advice if I'm making a noob mistake.